# Clustering

## We considered to use clinical trials clustering as the basic algorithm for our search application, so these file documents some experiments. At the end, we decided to use a different approach (based on term frequencies and tf-idf based cosine distance), because it fitted better the requirements of our project. Mainly, the requirement to present frequent terms that also cover large part of the dataset. So, again, this file documents only some experiments. 

The goal of this step is to present the users with a set of key terms for the data and allow them to select interesting tags interactively to narrow the results.

The process can be divided into two stages:

1. Data preprocessing
2. User interaction

Stage (1) comprises two main procedures. First, the terms are transformed into *term vectors*, where each column corresponds to each term in the dataset and each row corresponds to each document. The elements are then assigned weights; these can be either a simple binary measures (1 - the term is present in the given document; 0 - the term does not appear in the document) or statistical ranks of importance of the given term to the given document (such as TF-IDF or G2).

    Example: for the documents ["Mary had", "a little lamb"], term vectors with a binary measure would be (let's assume the terms are sorted in alphabatical order):

    [[0, 1, 0, 0, 1], #Document 1
    [1, 0, 1, 1, 0]]
    
The second step of data preprocessing is to calculate, for each document, its K nearest neighbours. These will be used to find the most relevant terms for display after the user selects a tag at the user interaction stage.

Stage (2) is user interaction. Initially, a set of highest-ranking terms for the entire dataset (calculated on the basis of the term vectors) is displayed to the user along with an input field. Once the user selects a term from the list, documents containing the term are selected. For each of the matched document, its neighbours are selected from the nearest neighbour list. Finally, the terms of the matched documents and neghbours are used to generate a more focused input list for user selection and the process is repeated.

In [1]:
import os
from sklearn.neighbors import NearestNeighbors
from math import log
import pandas as pd
import numpy as np
import pickle

In [2]:
# Set working directory as the current directory of the ipython notebook
working_dir = os.getcwd()
data_dir = os.path.join(working_dir, 'data')
print("Data directory: %s" % data_dir)

Data directory: C:\Study\CS102\project\project2\repro\CS109Project\data


Read in the serialized results of MetaMap processing.

In [3]:
df = pd.read_pickle(os.path.join(data_dir, 'mm.pckl'))

df.head()

,ngram,nct_id,criteria_id,ngram_index,score,term,cui,stype,cid
0,"((time, NN),)",NCT00001149,0,0,8.34,Time,C0040223,tmco,[G01.910]
1,"((uncontrolled, VBN), (seizure, NNS), (at, IN))",NCT00001149,0,8,16.21,Seizures,C0036572,sosy,"[C10.228.140.490.631, C10.597.742, C23.888.592..."
2,"((of, IN), (seizure, NNS), (during, IN))",NCT00001149,0,10,16.21,Seizures,C0036572,sosy,"[C10.228.140.490.631, C10.597.742, C23.888.592..."
3,"((seizure, NNS), (at, IN), (the, DT))",NCT00001149,0,11,16.21,Seizures,C0036572,sosy,"[C10.228.140.490.631, C10.597.742, C23.888.592..."
4,"((pattern, NN), (of, IN), (seizure, NNS))",NCT00001149,0,12,16.07,Seizures,C0036572,sosy,"[C10.228.140.490.631, C10.597.742, C23.888.592..."


# Calculate term vectors and document clusters

Generating clusters in done in two steps.

1. First, for each document, rank the set of terms for the document with G2 scores. 
2. Then, for each document, generate clusters of K most similar documents.

By default the term ranking stage uses MetaMap CUIs for terms; this can be configured as a parameter to the respective functions.

In [4]:
def g2(a, b, c, d):
    """ Calculate Dunning's log-likelihood score

    :param a: term frequency in corpus 1
    :type a: double
    :param b: term frequency in corpus 2
    :type b: double
    :param c: corpus 1 frequency of other terms (size - a)
    :type c: double
    :param d: corpus 2 frequency of other terms (size - b)
    :type d: double
    :return: log-likelihood score
    :rtype: double
    """
    a = a + 1.0
    b = b + 1.0
    c = c + 1.0
    d = d + 1.0
    G2 = 2 * (
        a * log(a) + b * log(b) + c * log(c) + d * log(d) - (a + b) * log(a + b) - (a + c) * log(a + c) - (b + d) * log(
            b + d) - (c + d) * log(c + d) + (a + b + c + d) * log(a + b + c + d))
    return G2

In [5]:
def term_frequencies(data, column='cui'):
    """ Returns the frequency map for the given column (defaults to 'cui'). 
    Used to generate term frequencies for the whole dataset, which are consulted in term score generation.
    
    :param data: MetaMap-tagged dataframe
    :type data: DataFrame
    :param column: column containing the terms whose frequencies are to be generated
    :type column: str
    """
    return data[column].value_counts()

In [6]:
def aggregate_data(data, column='cui', key='nct_id'):
    """ Groups rows sharing the same key (defaults to 'nct_id') 
    and collapses values of the specified column (defaults to 'cui') into a list.
    
    Used to transform MetaMap-tagged data (one row per MetaMap term) into a document-term matrix.
    
    :param data: MetaMap-tagged dataframe
    :type data: DataFrame
    :param column: the column whose values to collapse into a list
    :type column: str
    :param key: the column to group the data by
    :type key: str
    """
    grouped = data.groupby(data[key])
    aggregated = grouped.agg({column: lambda x: x.tolist()})
    return aggregated

In [7]:
def rank_terms(freq_map, aggregated, column='cui'):
    """ Ranks the terms in the given column (defaults to 'cui') with G2 similarity.
    
    :param freq_map: the dataset frequency map calculated by "term_frequencies"
    :type freq_map: Series
    :param aggregated: the data converted to a document-term matrix, generated by "aggregate_data"
    :type aggregated: DataFrame
    :param columm: the name of the column containing the data
    :type column: string
    """
    def calc_freqs(r):
        k,v = np.unique(r, return_counts=True)
        return dict(zip(k,v))

    scores = [] # 2D matrix: 1 document per row, 1 term per column
    corpus_size = freq_map.sum()
    for row in aggregated[column].get_values():
        doc_freqs = calc_freqs(row)
        doc_size = sum(doc_freqs.values())
        doc_vector = []
        for term, freq in freq_map.iteritems():
            a = int(doc_freqs[term]) if term in doc_freqs else 0
            b = freq
            c = doc_size - a
            d = corpus_size - b
            score = g2(a,b,c,d)
            doc_vector.append(score)
        scores.append(doc_vector)
    return np.array(scores)

In [8]:
def generate_clusters(term_vectors, k=50):
    """ Generates k nearest neighbours for the provided document-term score matrix.
    Outputs a matrix of distances and a matrix of indices of the neighbours in the aggregated data set.
    Both matrices have one row per document, one column per neighbour.
    
    :param term_vectors: the term scores per document calculated by "rank_terms"
    :type term_vectors: ndarray
    :param k: the number of nearest neighbours to generate
    :type k: int
    """
    real_k = min(k, len(term_vectors))
    nn = NearestNeighbors(n_neighbors=real_k, algorithm='ball_tree').fit(term_vectors)
    distances, indices = nn.kneighbors(term_vectors)
    return distances, indices

*term_frequencies(data_frame)* returns the counts for each term (CUI) across the whole dataset.

In [9]:
freq_map = term_frequencies(df)
freq_map.head()

C0030705    11009
C0036572     9953
C0019664     7490
C0019665     7490
C0012634     6862
Name: cui, dtype: int64

*aggregate_data(data_frame)* groups the MetaMap-tagged data (one row per one MM tag) according to the NCT ID of the original document.

In [10]:
agg_df = aggregate_data(df)
agg_df.head()

,cui
nct_id,
NCT00001149,"[C0040223, C0036572, C0036572, C0036572, C0036..."
NCT00001192,"[C0030705, C3661466, C0042960, C0033927, C0033..."
NCT00001205,"[C0021081, C0039798, C0030705, C0008059, C0001..."
NCT00001218,"[C0031206, C0018684, C0031206, C0018684, C1708..."
NCT00001262,"[C0012634, C0235031, C0012634, C0021270, C0235..."


*rank_terms(frequency_map, aggregated_terms)* calculates the scores for each term per each document. It returns a list where each row corresponds to each document in *aggregated* and each column corresponds to each term in *frequency_map*.

In [11]:
term_vectors = rank_terms(freq_map, agg_df)
print term_vectors[:10]

[[  1.48128593e+01   3.73000780e+01   1.30281259e+01 ...,   1.21529518e+01
    1.21529518e+01   1.21529518e+01]
 [  8.26224573e-02   8.41583529e-01   1.00962763e+01 ...,   1.30425651e+01
    1.30425651e+01   1.30425651e+01]
 [  7.58073218e+00   1.36250735e+01   9.11352458e+00 ...,   1.00081409e+01
    1.00081409e+01   1.00081409e+01]
 ..., 
 [  5.22303822e-01   3.55070137e-01   6.95985006e-02 ...,   1.35502672e+01
    1.35502672e+01   1.35502672e+01]
 [  1.16574127e+00   1.12555932e+01   7.40998322e+00 ...,   1.02879057e+01
    1.02879057e+01   1.02879057e+01]
 [  1.09748542e-03   3.07013192e+00   1.15059729e-01 ...,   1.47742748e+01
    1.47742748e+01   1.47742748e+01]]


*generate_clusters(term_vectors, k_neighbours)* calculates *k* nearest neighbours for each document. It returns (1) a list of distances between each document and its neighbours (the smaller the better) and (2) a list of indices of documents in the original data frame.

In [14]:
cluster_distances, cluster_indices = generate_clusters(term_vectors, k=10)
print cluster_distances, cluster_indices
print np.mean(cluster_distances), np.std(cluster_distances)

[[   0.           93.31178852   93.66380314 ...,  100.75898977  101.4364923
   103.48011933]
 [   0.          145.83035197  153.42381646 ...,  156.69696465  157.7790185
   157.85909544]
 [   0.          319.26218212  328.75135917 ...,  359.57536786
   360.15883844  360.17595347]
 ..., 
 [   0.          244.77781111  246.4852524  ...,  252.02956025
   252.70038576  252.9716634 ]
 [   0.          172.9420408   182.72826205 ...,  187.12053273
   187.73843008  187.81298789]
 [   0.          259.93173628  260.66232062 ...,  263.53621165  263.9977189
   264.03799299]] [[   0  126  179 ...,  141  295  444]
 [   1  568  295 ..., 1153 1200  289]
 [   2  227  311 ...,  197  194  152]
 ..., 
 [1333  685  192 ...,  194  152  196]
 [1334  192  441 ...,  196 1069 1281]
 [1335  194  390 ...,  246  171  219]]
169.923166059 107.409757613


### Utility functions to run the full clustering pipeline and serialize the intermediate and final results so they can be used later by other modules

In [15]:
def clustering_pipeline(data, data_column='cui', aggregate_key='nct_id', k=10):
    """ Wraps up the individual clustering steps (frequency calculation, term ranking, etc.) in a processing pipeline.
    Returns the frequency map, aggregated data, term scores, neighbour distances and neighbour indices.
    
    :param data: the original MetaMap-tagged data to score and cluster
    :type data: DataFrame
    :param data_column: the column in the original dataframe containing the terms to score
    :type data_column: str
    :param aggregate_key: the column in the original dataframe to aggregate the data by
    :type aggregate_key: str
    :param k: the number of neighbours to find for each document
    :type k: int
    """
    freq_map = term_frequencies(data)
    agg_df = aggregate_data(data)
    term_vectors = rank_terms(freq_map, agg_df)
    cluster_distances, cluster_indices = generate_clusters(term_vectors)
    return freq_map, agg_df, term_vectors, cluster_distances, cluster_indices

In [16]:
freq_map, agg_df, term_vectors, cluster_distances, cluster_indices = clustering_pipeline(df, 'cui', 'nct_id', 10)

In [17]:
def write_data(data_dir, freq_map, agg_df, term_vectors, cluster_distances, cluster_indices):
    print "Serializing freq_map to %s" % os.path.join(data_dir, 'clust_fm.pckl')
    freq_map.to_pickle(os.path.join(data_dir, 'clust_fm.pckl'))
    print "Serializing agg_df to %s" % os.path.join(data_dir, 'clust_agg.pckl')
    agg_df.to_pickle(os.path.join(data_dir, 'clust_agg.pckl'))
    print "Serializing term_vectors to %s" % os.path.join(data_dir, 'clust_tv.pckl')
    with open(os.path.join(data_dir, 'clust_tv.pckl'), 'wb') as fh:
        pickle.dump(term_vectors, fh)
    print "Serializing cluster_distances to %s" % os.path.join(data_dir, 'clust_cd.pckl')
    with open(os.path.join(data_dir, 'clust_cd.pckl'), 'wb') as fh:    
        pickle.dump(cluster_distances, fh)
    print "Serializing cluster_indices to %s" % os.path.join(data_dir, 'clust_ci.pckl')
    with open(os.path.join(data_dir, 'clust_ci.pckl'), 'wb') as fh:    
        pickle.dump(cluster_indices, fh)

In [18]:
write_data(data_dir, freq_map, agg_df, term_vectors, cluster_distances, cluster_indices)

Serializing freq_map to C:\Study\CS102\project\project2\repro\CS109Project\data\clust_fm.pckl
Serializing agg_df to C:\Study\CS102\project\project2\repro\CS109Project\data\clust_agg.pckl
Serializing term_vectors to C:\Study\CS102\project\project2\repro\CS109Project\data\clust_tv.pckl
Serializing cluster_distances to C:\Study\CS102\project\project2\repro\CS109Project\data\clust_cd.pckl
Serializing cluster_indices to C:\Study\CS102\project\project2\repro\CS109Project\data\clust_ci.pckl
